In [1]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf

from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

import wavy

In [3]:
tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].Symbol

In [4]:
msft = yf.Ticker("MSFT")

# get stock info
msft.info

# get historical market data from 2005 on
hist = msft.history(period="max", start="2005-01-01")

In [5]:
hist.head(10)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2005-01-03,19.040902,19.147476,18.934330,18.998274,65002900,0.0,0
2005-01-04,19.090628,19.254038,18.941426,19.069313,109442100,0.0,0
2005-01-05,19.069320,19.254045,19.012481,19.026691,72463500,0.0,0
2005-01-06,19.076418,19.225618,18.927216,19.005369,76890500,0.0,0
2005-01-07,19.055106,19.104839,18.913010,18.948534,68723300,0.0,0
2005-01-10,18.898802,19.083527,18.856173,19.040897,70376600,0.0,0
2005-01-11,18.962741,19.055103,18.905902,18.991159,64712000,0.0,0
2005-01-12,19.019586,19.076425,18.913015,19.026691,72940600,0.0,0
2005-01-13,18.955640,19.040897,18.586190,18.664343,89861600,0.0,0


In [6]:
hist[['Open', 'High', 'Low', 'Close']].plot()

In [7]:
hist.Dividends.plot()

In [8]:
hist['Stock Splits'].plot()

In [9]:
hist = hist[['Open', 'High', 'Low', 'Close']]

In [10]:
hist = hist.pct_change()

In [11]:
hist.Close.plot()

In [12]:
print(len(hist) - len(hist.dropna()))
hist.dropna(inplace=True)

1


In [13]:
hist.High.hist(bins=200)

In [14]:
x, y = wavy.create_panels(hist, lookback=10, horizon=1)

In [15]:
y[0]

,Open,High,Low,Close
Date,,,,
2005-01-19,0.006915,-0.003415,0.003096,-0.012918


In [16]:
# predict only the highest price
y = y[['High']]

In [17]:
# try single col inputs first
x_high = x[['High']]

In [18]:
# constant = wavy.ConstantModel(x_high, y, model_type="regression")
# constant.fit()

In [19]:
baseline = wavy.BaselineModel(x_high, y, model_type="regression")
baseline.fit() # val_MAE: 0.0136

100%|██████████| 430/430 [00:00<00:00, 112827.70it/s]
2022-02-24 00:35:45.316474: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


97/97 [==============================] - 1s 4ms/step - loss: 4.1137e-04 - MAE: 0.0135 - val_loss: 3.8832e-04 - val_MAE: 0.0136


In [20]:
# How to interpret that error? What does it mean comparing to the mean/std of the val set?
np.median(baseline.y_val.flatten())

0.0015305177595862363

In [21]:
lr = wavy.LinearRegression(x_high, y)
lr.fit(epochs=20)

100%|██████████| 430/430 [00:00<00:00, 99150.67it/s]


Epoch 1/20
97/97 [==============================] - 1s 4ms/step - loss: 6.2304e-04 - MAE: 0.0178 - val_loss: 4.7363e-04 - val_MAE: 0.0161
Epoch 2/20
97/97 [==============================] - 0s 2ms/step - loss: 5.3291e-04 - MAE: 0.0164 - val_loss: 4.1289e-04 - val_MAE: 0.0150
Epoch 3/20
97/97 [==============================] - 0s 2ms/step - loss: 4.6190e-04 - MAE: 0.0152 - val_loss: 3.5928e-04 - val_MAE: 0.0140
Epoch 4/20
97/97 [==============================] - 0s 2ms/step - loss: 3.9975e-04 - MAE: 0.0141 - val_loss: 3.2318e-04 - val_MAE: 0.0132
Epoch 5/20
97/97 [==============================] - 0s 2ms/step - loss: 3.5518e-04 - MAE: 0.0132 - val_loss: 2.8637e-04 - val_MAE: 0.0123
Epoch 6/20
97/97 [==============================] - 0s 3ms/step - loss: 3.1797e-04 - MAE: 0.0124 - val_loss: 2.6556e-04 - val_MAE: 0.0118
Epoch 7/20
97/97 [==============================] - 0s 3ms/step - loss: 2.9011e-04 - MAE: 0.0118 - val_loss: 2.4033e-04 - val_MAE: 0.0111
Epoch 8/20
97/97 [================

In [22]:
y.test[0]

,High
Date,
2020-06-11,-0.013903


In [23]:
y.test[1]

,High
Date,
2020-06-12,-0.020637


In [24]:
baseline.x_test.shape

(429, 1, 1)

In [ ]:
from copy import deepcopy

def update(panel, other):
    panel = deepcopy(panel)

    for i, j in zip(panel, other):
        i.iloc[:,:] = j
    return panel

In [ ]:
y.test.shape

In [ ]:
a = update(y.test[1:], baseline.predict())

In [ ]:
a[0]

In [ ]:
y.test[1]

In [ ]:
copy(y.test)

In [28]:
b = baseline.predict()

In [29]:
b[0]

,High
Date,
2020-06-12,-0.013903


In [ ]:
dense = wavy.DenseModel(x_high, y, model_type="regression")
dense.fit(epochs=20)

In [ ]:
conv = wavy.ConvModel(x_high, y, model_type="regression")
conv.fit(epochs=20)

# using a cannon ball to kill an ant!

In [ ]:
dummy = wavy.ShallowModel(x_high, y, model=DummyRegressor, metrics=[mean_absolute_error], strategy="mean")
dummy.fit()

In [ ]:
rf = wavy.ShallowModel(x, y, model=RandomForestRegressor, metrics=[mean_absolute_error], n_estimators=50)
rf.fit()

In [ ]:
dense = wavy.DenseModel(x, y, model_type="regression")
dense.fit(epochs=20)

In [ ]:
# Try the same models but with all input columns
# TODO: Use function to compute default results per model

# lr = wavy.LinearRegression(x, y)
# lr.fit(epochs=20)